In [80]:

import os
import json
import pandas as pd
import numpy as np
import pickle as pkl
import seaborn as sns

import torch

import torch.nn as nn
import torch.nn.functional as F
# load config
with open('config.json', 'r') as f:
    config = json.load(f)
cwd = os.getcwd()
os.chdir(config['REPODIR'])
import Utils as U
# from Corpus import Corpus
os.chdir(cwd)

from collections import Counter, defaultdict
import itertools


from tqdm.auto import tqdm, trange
from collections import Counter
import random
from torch import optim
from torch.nn.utils.rnn import pad_sequence
from torch.utils.tensorboard import SummaryWriter
from gensim.test.utils import common_texts
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="dark")
import pickle as pkl
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
import nltk
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataloader import default_collate

torch.set_default_dtype(torch.float64)

suffix = "full"

In [5]:
with open('embedding_data_final.pkl', 'rb') as f:
    embed = pkl.load(f)

embed_df = pd.DataFrame(embed)

In [6]:
embed_df = embed_df.rename(columns = {0: 'seqid', 1: 'passage_key', 2: 'sent_embeddings'})


In [7]:
data = U.load_file('data_vFFFF.pkl', 'pkl', config['DATADIR'])

In [8]:
import gensim

import gensim.downloader as api
gensim_model = api.load("glove-wiki-gigaword-300") 



In [9]:
embedding_weights = gensim_model.vectors


In [10]:


embedding_dim = gensim_model.vector_size
num_embeddings = embedding_weights.shape[0]

embedding_layer = nn.Embedding(num_embeddings, embedding_dim)
embedding_layer.weight.data.copy_(torch.from_numpy(embedding_weights))

tensor([[ 0.0466,  0.2132, -0.0074,  ...,  0.0091, -0.2099,  0.0539],
        [-0.2554, -0.2572,  0.1317,  ..., -0.2329, -0.1223,  0.3550],
        [-0.1256,  0.0136,  0.1031,  ..., -0.3422, -0.0224,  0.1368],
        ...,
        [ 0.0757, -0.0405,  0.1834,  ...,  0.2184,  0.3097,  0.4376],
        [ 0.8145, -0.3622,  0.3119,  ...,  0.0755,  0.2841, -0.1756],
        [ 0.4292, -0.2969,  0.1501,  ...,  0.2898,  0.3262, -0.0591]])

In [11]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [12]:
data_df = pd.DataFrame(data)
data_df.head()

data_df['joined_text'] = data_df['text'].apply(' '.join)

In [13]:
text = data_df['joined_text'].to_numpy()
text_clean = [tokenizer.tokenize(t.lower()) for t in text]

In [14]:
word2index = {token: token_index for token_index, token in enumerate(gensim_model.index_to_key)}
index2word = {val:key for key, val in word2index.items()}

In [15]:
tokenized_sequences = [[word2index[word] for word in text if word in word2index ] for text in text_clean]

In [16]:
lengths = [len(seq) for seq in tokenized_sequences]
max_len = max(lengths)
min_len = min(lengths)

In [17]:
test = tokenized_sequences[0]
# torch.IntTensor(tokenized_sequences)

data_df['sequences'] = tokenized_sequences

In [ ]:
# embed_df = embed_df.dropna(subset = ['author_id', 'sent_embedding'])

In [18]:

n_classes = data_df.author_id.nunique()

from sklearn.preprocessing import OneHotEncoder
# label_encoder=OneHotEncoder(sparse_output=False)
label_encoder=OneHotEncoder()


In [38]:
y= label_encoder.fit_transform(data_df['author_id'].to_numpy(dtype='int32').reshape(-1,1))
y = y.toarray()
X = data_df['sequences'].apply(np.array)

In [39]:
X = [(toop[0], toop[1]) for toop in list(zip(X, lengths))]

In [40]:
X[0]

(array([    84,     41,     43,    987,     20,      3,     81,     84,
        171390,  50628,    119,     36,  28379,      0,    338,   6712,
            34,      0,    794,     15,      7,   4503,     48,     20,
            15,     36,     22,     50,    196,     19,     22,    729,
           727,   2094,     17,      0,   1451,      3,      0,  29644,
            19,     17,      0,   1451,      3,      0,     60,      4,
            37,     79,    308,     40,  41831,    420,    359,   9139,
           114,    308,    848,    149,     77,    889,    390,     30,
           238,      4,   1007,      0,   2353,     13,      7,   2377,
          2133,     21,  10056,     42,     54,     33,     51, 136953,
          3606,     36,    134,   3428,      3,    134,    339,      6,
          1276,     20,   1706,     12, 171390,  50628,     40,     29,
          1159,     10,     18,     16,      4,      0,   1451,    143,
            43,     81,   1619,    285,      4,   9496,     84, 

In [53]:
def pad_with(vector, pad_width, iaxis, kwargs):
    pad_value = kwargs.get('padder', 0)
    vector[:pad_width[0]] = pad_value
    vector[-pad_width[1]:] = pad_value

In [76]:
MAX_LENGTH = 512

X_trunc = []

for x in X:
    print(x[1])
    deficit = MAX_LENGTH - x[1]
    if deficit > 0:
        X_trunc.append(x[0])
    else:
        X_trunc.append(x[0][:MAX_LENGTH])


332
434
466
518
353
393
247
459
416
282
470
326
332
361
508
348
335
277
424
432
357
402
472
416
342
377
270
505
367
460
546
484
535
443
545
506
450
605
549
495
470
487
531
538
444
572
471
478
426
575
518
555
552
474
529
523
497
557
519
496
483
500
463
606
564
435
569
437
418
517
490
505
612
517
533
450
457
596
395
518
580
477
526
312
472
474
476
537
401
567
549
448
569
453
524
508
550
646
529
549
559
511
397
511
553
470
458
486
548
570
577
461
492
489
524
481
486
536
527
487
397
542
496
517
470
465
436
548
474
342
583
549
462
455
512
456
407
469
446
420
419
503
332
491
509
520
411
570
528
528
488
412
461
473
374
497
392
426
507
486
478
487
532
476
499
607
371
505
422
531
579
541
473
411
493
552
444
355
407
372
493
422
346
493
440
512
433
337
518
476
537
332
534
506
352
440
470
461
459
528
501
556
455
475
467
393
464
439
485
508
443
535
375
399
408
463
365
353
484
368
425
430
581
450
415
340
527
583
424
479
421
545
475
535
303
455
418
433
551
374
389
319
553
418
377
612
544
532
506
472


In [77]:
X_trunc.sort(key=lambda x: len(x),reverse = True)

In [79]:
len(X_trunc[-1])

40

In [87]:
torch.tensor(X_trunc[0]).shape

tensor_list = [torch.tensor(x).unsqueeze(1) for x in X_trunc]

In [88]:
tensor_list[3].shape

torch.Size([512, 1])

In [93]:
X_t = pad_sequence([torch.tensor(x) for x in X_trunc], batch_first=True)


In [94]:
[xt.shape for xt in X_t]

[torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),


In [95]:
test_size = 0.2
val_size = 0.2
random_state =699

X_train, X_test, y_train, y_test = U.train_test_split(X_t, y, test_size=test_size,
                                                        random_state=random_state,
                                                        stratify=y)

# Split train set into train and validation sets
X_train, X_val, y_train, y_val = U.train_test_split(X_train, y_train, test_size=val_size/(1-test_size),
                                                    random_state=random_state,
                                                    stratify=y_train)

In [96]:
device = 'cpu'
# device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if (device == "cuda:0" or device == 'mps') else {}
collate_func = lambda x: tuple(x_.to(device) for x_ in default_collate(x)) if device != "cpu" else default_collate(x)

torch.Size([16671, 512])

In [100]:
class DocumentAttentionClassifier(nn.Module):
    
    def __init__(self, vocab_size, embedding_size, num_heads, hidden_dim, trained_weights, n_classes):
        '''
        Creates the new classifier model. embeddings_fname is a string containing the
        filename with the saved pytorch parameters (the state dict) for the Embedding
        object that should be used to initialize this class's word Embedding parameters
        '''
        super(DocumentAttentionClassifier, self).__init__()
        
        # Save the input arguments to the state
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.num_heads = num_heads
        self.embeddings_fname = vocab_size        
        
        self.embeddings = nn.Embedding.from_pretrained(trained_weights, freeze = False)
        # self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size, hidden_dim, batch_first=True, bidirectional=True)
        self.attention = nn.Linear(hidden_dim * 2, 1)
        self.fc = nn.Linear(hidden_dim * 2, n_classes)
        # trained_weights = torch.load(embeddings_fname)['target_embeddings.weight']

        
        # # self.embeddings = nn.Embedding()

        # self.lstm = nn.LSTM(embedding_size, hidden_dim, bidirectional=True)
        # # self.attention = torch.rand(self.num_heads, self.embedding_size, requires_grad = True, device=device)
        # self.attention = torch.rand(self.num_heads, hidden_dim * 2, requires_grad = True, device=device)
        # self.linear = nn.Linear(num_heads * embedding_size, n_classes)
    
    def forward(self, text):
       
        w = self.embeddings(text)

        lstm_output, _ = self.lstm(w.transpose(1,2)) 
        attention_scores = self.attention(lstm_output) 
        attention_weights = F.softmax(attention_scores, dim=1)  
        context_vector = torch.sum(attention_weights * lstm_output, dim=1)  
        output = self.fc(context_vector)  # shape: (batch_size, num_classes)
        # w = w.squeeze()

        # lstm_out, _ = self.lstm(w.T)
        # # w = torch.t(self.embeddings(word_ids).squeeze()) # Embedding_Dim 
        # r = torch.matmul(self.attention, lstm_out.T)
        # a = torch.softmax(r, 1)
        # reweighted = a @ w.T
        # output = self.linear(reweighted.view(-1))

        # return torch.softmax(output, dim=0), a.T
        return output, attention_weights

In [101]:
datasets = {}

datasets['train'] = list(zip(X_train, y_train))
datasets['val'] = list(zip(X_val, y_val))
datasets['test'] = list(zip(X_test, y_test))

train_list = datasets['train']
val_list = datasets['val']
test_list = datasets['test']

model = DocumentAttentionClassifier(1, 500, 4, 32, torch.Tensor(embedding_weights), n_classes)

In [102]:
def run_eval(model, eval_data, n_classes, kwargs):
    '''
    Scores the model on the evaluation data and returns the F1
    Eval Data must be in DataLoader-ready format
    '''

    eval_loader = DataLoader(eval_data, batch_size = 1, shuffle = False, collate_fn=collate_func, **kwargs)

    threshold = 0.2
    probs = np.zeros((len(eval_loader), n_classes))
    labels = []
    
    with torch.no_grad():
        for idx, x in enumerate(eval_loader):
            word_ids, label = x
            labels.append(label.cpu().numpy())
            output, weights = model(word_ids.double())
            probs[idx] = output.cpu().numpy()
    
    
    y_pred = np.array([np.argmax(p) for p in probs], dtype = int)
    labels = np.array(labels)

    y_true = [np.argmax(l) for l in labels]
    
    
    return labels, y_pred, f1_score(y_true, y_pred, average='weighted')

In [103]:

loss_period = 5
# model = model.to(device)
writer = SummaryWriter()
loss_function = nn.CrossEntropyLoss()

# VVV GOLD STANDARD VVV
optimizer = optim.AdamW(model.parameters(), lr = 5e-2, weight_decay = 0.001)
# ^^^ GOLD STANDARD ^^^

# optimizer = optim.AdamW(model.parameters(), lr = 5e-3, weight_decay = 0.001)

# optimizer = optim.AdamW(model.parameters())
# optimizer = optim.RMSprop(model.parameters(), 5e-3)
# optimizer = optim.SGD(model.parameters(), lr = 5e-4)

train_loader = DataLoader(train_list, batch_size=32, shuffle=True, collate_fn=collate_func, **kwargs)
n_epochs = 1

# + vscode={"languageId": "python"}
loss_idx = 0
loss_record = []
model.train()

# + vscode={"languageId": "python"}r
for epoch in tqdm(range(n_epochs)):

    loss_sum = 0

    for step, data in tqdm(enumerate(train_loader)):

        word_ids, labels = data
        labels = labels.argmax(-1)
        model.train()
        model.zero_grad()
        output, weights = model(word_ids)
        loss = loss_function(output, labels)
        loss.backward()
        optimizer.step()

        loss_sum += loss.item()
        
        # TODO: Based on the details in the Homework PDF, periodically
        # report the running-sum of the loss to tensorboard. Be sure
        # to reset the running sum after reporting it.

        if not step % loss_period and step:
            writer.add_scalar("Loss", loss_sum, loss_idx)
            if not step % (loss_period * 10) and step:
                model.eval()
                _y, _y2, f1 = run_eval(model, val_list, n_classes, kwargs)
                writer.add_scalar("F1", f1, loss_idx)
                model.train()
            loss_record.append(loss_sum)
            loss_sum = 0
            loss_idx += 1

        # TODO: it can be helpful to add some early stopping here after
        # a fixed number of steps (e.g., if step > max_steps)
        

# once you finish training, it's good practice to switch to eval.
model.eval()

y_true, y_pred, f1 = run_eval(model, val_list, n_classes, kwargs)
print("Eval F1 Score of : "+ str(f1))

0it [00:01, ?it/s]/1 [00:00<?, ?it/s]
  0%|          | 0/1 [00:01<?, ?it/s]


RuntimeError: input.size(-1) must be equal to input_size. Expected 500, got 512

In [ ]:
y_true, y_pred, f1 = run_eval(model, test_list, n_classes, kwargs)
print("Test F1 Score of : "+ str(f1))

In [ ]:
torch.save(optimizer.state_dict(), 'trained_opt_author' + suffix)
torch.save(model.state_dict(), 'trained_model_author' + suffix)